In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

e:\User\PC\AI_project\AI_final


In [2]:
from libs.common import *


In [3]:
load_dotenv(find_dotenv())


True

In [1]:
# os.getenv('POSTGRESQL_DBMS_KEY')


In [5]:
db = SQLDatabase.from_uri(os.getenv('POSTGRESQL_DBMS_KEY'))
print(db)


In [6]:
print(db.dialect)
print(db.get_usable_table_names())











postgresql
['company_events', 'company_news', 'company_profiles', 'company_ratio_finance']


In [7]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=os.getenv('OPEN_API_KEY'))


In [11]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Năm 2024 có những sự kiện gì nổi bật đối với công ty mã A32"})
print(response)

SELECT "title", "source", "publish_date"
FROM company_news
WHERE ticker = 'A32' AND publish_date >= '2024-01-01' AND publish_date < '2025-01-01'
ORDER BY publish_date DESC
LIMIT 5;


In [12]:
db.run(response)


"[('A32: Ngày đăng ký cuối cùng trả cổ tức bằng tiền mặt', 'TCBS', '2024-08-09 10:53:00'), ('A32: Thông báo ngày đăng ký cuối cùng thực hiện trả cổ tức còn lại năm 2023 bằng tiền', 'TCBS', '2024-08-07 15:43:00'), ('A32: Nghị quyết Hội đồng quản trị', 'TCBS', '2024-08-07 15:42:00'), ('A32: Nghị quyết Đại hội đồng cổ đông thường niên năm 2024', 'TCBS', '2024-08-07 10:44:00'), ('A32: Điều lệ tổ chức và hoạt động sửa đổi lần thứ 6', 'TCBS', '2024-08-06 14:49:00')]"

In [ ]:
chain.get_prompts()[0].pretty_print()


In [13]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "Năm 2024 có những sự kiện gì nổi bật đối với công ty mã A32"})

"[('A32: Ngày đăng ký cuối cùng trả cổ tức bằng tiền mặt', 'TCBS', '2024-08-09 10:53:00'), ('A32: Thông báo ngày đăng ký cuối cùng thực hiện trả cổ tức còn lại năm 2023 bằng tiền', 'TCBS', '2024-08-07 15:43:00'), ('A32: Nghị quyết Hội đồng quản trị', 'TCBS', '2024-08-07 15:42:00'), ('A32: Nghị quyết Đại hội đồng cổ đông thường niên năm 2024', 'TCBS', '2024-08-07 10:44:00'), ('A32: Điều lệ tổ chức và hoạt động sửa đổi lần thứ 6', 'TCBS', '2024-08-06 14:49:00')]"

In [14]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "Năm 2024 có những sự kiện gì nổi bật đối với công ty mã A32"})

'Trong năm 2024, các sự kiện nổi bật đối với công ty mã A32 bao gồm:\n1. Ngày đăng ký cuối cùng trả cổ tức bằng tiền mặt\n2. Thông báo ngày đăng ký cuối cùng thực hiện trả cổ tức còn lại năm 2023 bằng tiền\n3. Nghị quyết Hội đồng quản trị\n4. Nghị quyết Đại hội đồng cổ đông thường niên năm 2024\n5. Điều lệ tổ chức và hoạt động sửa đổi lần thứ 6\n\nCác thông tin này được đăng trên trang web TCBS vào các ngày trong tháng 8 năm 2024.'

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)